In [2]:
import pandas as pd
import numpy as np
from env import api_key
import requests
import json
from acquire import *
from bs4 import BeautifulSoup
import os
from prepare import *

# Acquire

In [ ]:
# function calling the acquired df
links = get_links_to_bills()

In [ ]:
df = acquire_bills(links)

# Prepare

In [ ]:
bills_df = prep_bills(df)
bills_df

In [ ]:
# creating a lemmatized column and cleaning the df
bills_df['lem']= bills_df.bill_text.apply(clean_text)

In [ ]:
bills_df['model']= bills_df.lem.apply(join)
bills_df.head()

In [ ]:
# splitting the data
train, X_train, y_train, X_val, y_val, X_test, y_test = prepare.split_data(new_df, 'model')